Notebook to use the helper_CI tools to build the QED-CIS matrix using
$$ <ij||kl> + D_{ij||kl} + T + V + Q + d $$

Note we might want to look at these equations [here](https://www.chem.fsu.edu/~deprince/programming_projects/cis/).
Ideally, this will yield the same result as ../simple_cis.ipynb.

In [1]:
import time
import numpy as np
np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_cqed_rhf import cqed_rhf
# Compute size of Hamiltonian in GB
from scipy.special import comb
# prepare the determinant list
from helper_CI import Determinant, HamiltonianGenerator, compute_excitation_level
from itertools import combinations

In [2]:
def spin_idx_to_spat_idx_and_spin(P):
    """ function to take the numeric label of a spin orbital
        and return the spatial index and the spin index separately.
        Starts counting from 0:
        
        Arguments
        ---------
        P : int
            spin orbital label
        
        Returns
        -------
        [p, spin] : numpy array of ints
            p is the spatial orbital index and spin is the spin index.
            spin = 1  -> alpha
            spin = -1 -> beta
            
        Example
        -------
        >>> spin_idx_to_spat_idx_and_spin(0)
        >>> [0, 1]
        >>> spin_idx_to_spat_idx_and_spin(3)
        >>> [1, -1]
        
    """
    spin = 1
    if P % 2 == 0:
        p = P / 2
        spin = 1
    else:
        p = (P-1) / 2
        spin = -1
    return np.array([p, spin], dtype=int)


def map_spatial_to_spin(tei_spatial, I, J, K, L):
    """ function to take two electron integrals in the spatial orbital basis
        in chemist notation along with 4 indices I, J, K, L and return
        the corresponding two electron integral in the spin orbital basis
        in phycisit notation, <IJ||KL>
    
    """
    # Phys to Chem: <IJ||KL> -> [IK|JL] - [IL|JK]
    i_s = spin_idx_to_spat_idx_and_spin(I)
    k_s = spin_idx_to_spat_idx_and_spin(K)
    j_s = spin_idx_to_spat_idx_and_spin(J)
    l_s = spin_idx_to_spat_idx_and_spin(L)
    
    #print(i_s[1])
    # (ik|jl)
    spat_ikjl = tei_spatial[i_s[0], k_s[0], j_s[0], l_s[0]] * ( i_s[1] == k_s[1] ) *  ( j_s[1] == l_s[1] )
    
    # (il|jk)
    spat_iljk = tei_spatial[i_s[0], l_s[0], j_s[0], k_s[0]] * ( i_s[1] == l_s[1] ) *  ( j_s[1] == k_s[1] )
    
    return spat_ikjl - spat_iljk

def map_spatial_dipole_to_spin(mu, I, J, K, L):
    """ function to take two electron integrals in the spatial orbital basis
        in chemist notation along with 4 indices I, J, K, L and return
        the corresponding two electron integral in the spin orbital basis
        in phycisit notation, <IJ||KL>
    
    """
    # Phys to Chem: <IJ||KL> -> [IK|JL] - [IL|JK]
    i_s = spin_idx_to_spat_idx_and_spin(I)
    k_s = spin_idx_to_spat_idx_and_spin(K)
    j_s = spin_idx_to_spat_idx_and_spin(J)
    l_s = spin_idx_to_spat_idx_and_spin(L)
    
    #print(i_s[1])
    # (ik|jl)
    spat_ikjl = mu[i_s[0], k_s[0]] * mu[j_s[0], l_s[0]] * ( i_s[1] == k_s[1] ) *  ( j_s[1] == l_s[1] )
    
    # (il|jk)
    spat_iljk = mu[i_s[0], l_s[0]] * mu[j_s[0], k_s[0]] * ( i_s[1] == l_s[1] ) *  ( j_s[1] == k_s[1] )
    
    return spat_ikjl - spat_iljk

In [ ]:

# Check energy against psi4?
compare_psi4 = True

# Memory for Psi4 in GB
# psi4.core.set_memory(int(2e9), False)
psi4.core.set_output_file('output.dat', False)

# Memory for numpy in GB
numpy_memory = 2

mol_str = """
H 
H 1 1.0
symmetry c1
"""

options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8
                  }



# photon energy
omega_val = 4.75 / psi4.constants.Hartree_energy_in_eV

# define the lambda vector
lambda_vector = np.array([0.1, 0.1, 0.1])

mol = psi4.geometry(mol_str)


psi4.set_options(options_dict)

print('\nStarting SCF and integral build...')
t = time.time()

# First compute SCF energy using Psi4
scf_e, wfn = psi4.energy('SCF', return_wfn=True)

# now compute cqed-rhf to get transformation vectors with cavity
cqed_rhf_dict = cqed_rhf(lambda_vector, mol_str, options_dict)

# grab necessary quantities from cqed_rhf_dict
rhf_reference_energy = cqed_rhf_dict["RHF ENERGY"]
cqed_reference_energy = cqed_rhf_dict["CQED-RHF ENERGY"]
C = cqed_rhf_dict["CQED-RHF C"]
D = cqed_rhf_dict["CQED-RHF DENSITY MATRIX"]
eps = cqed_rhf_dict["CQED-RHF EPS"]
dc = cqed_rhf_dict["DIPOLE ENERGY (1/2 (\lambda \cdot <\mu>_e)^2)"]

# collect rhf wfn object as dictionary
wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# update wfn_dict with orbitals from CQED-RHF
wfn_dict["matrix"]["Ca"] = C
wfn_dict["matrix"]["Cb"] = C
# update wfn object
wfn = psi4.core.Wavefunction.from_file(wfn_dict)

# Grab data from wavfunction class
Ca = wfn.Ca()
ndocc = wfn.doccpi()[0]
nmo = wfn.nmo()



mints = psi4.core.MintsHelper(wfn.basisset())


Here we will build both the matrix of 1e integrals ($\langle p|\hat{O}_1|q \rangle$, where 
$\hat{O}_1$ contains electronic kinetic, nuclear-electron attraction, scaled quadrupole, and scaled dipole) in the AO basis so that they are (nao x nao) arrays, and we will build the $\langle pq||rs\rangle$ integrals augmented
by 2-electron scaled dipole integrals in the spin orbital 
basis which are (nso x nso x nso x nso) = (2 * nao x 2 * nao x 2 * nao x 2 * nao).

In [ ]:
# preparing 1e- and 2e- arrays
H = np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential()) + Q_PF + d_PF



print('\nTotal time taken for ERI integrals: %.3f seconds.\n' % (time.time() - t))

#Make spin-orbital MO
print('Starting AO -> spin-orbital MO transformation...')
t = time.time()
MO = np.asarray(mints.mo_spin_eri(Ca, Ca))

nso = 2 * nmo
TDI_spin = np.zeros((nso, nso, nso, nso))
for i in range(nso):
    for j in range(nso):
        for k in range(nso):
            for l in range(nso):
                TDI_spin[i, j, k, l] = map_spatial_dipole_to_spin(l_dot_mu_el_cmo, i, j, k, l)
                

MO += TDI_spin
    
print("Printing 1-e H matrix")   
print(H)
print(F'The dimensions of the 1-e H matrix is {H.shape}')
print(F'The dimesions of the 2-e integral arrays in the spin orbital basis is {MO.shape}')

Now we are going to transform 1-e integrals into the CQED-RHF basis

In [ ]:

# Update H, transform to MO basis and tile for alpha/beta spin
H = np.einsum('uj,vi,uv', Ca, Ca, H)


# prepare the g matrix by transforming the l_dot_mu_el matrix * sqrt(\omega / 2)
g_mat = np.einsum('uj,vi,uv', Ca, Ca, -np.sqrt(omega_val / 2) * l_dot_mu_el)

print(H)
print(g_mat)




Now we are going to put the 1-e arrays into the spin-orbital basis, so that each in the resulting
arrays is of the form $H_{pq} = \langle \chi_p | \hat{O}_1 | \chi_q \rangle$.  This array will be 
organized into spin blocks as follows:

\begin{equation}
\begin{bmatrix}
\langle p | \hat{O}_1 |q\rangle  & \langle p | \hat{O}_1| \overline{q}\rangle \\
\langle \overline{p} | \hat{O}_1|q\rangle  & \langle \overline{p}| \hat{O}_1| \overline{q}\rangle
\end{bmatrix}
\end{equation}

In [ ]:
H = np.repeat(H, 2, axis=0)
print("printing H after 1st repeat")
print(H)
H = np.repeat(H, 2, axis=1)
print("printing H after 2nd repeat")
print(H)
g_mat = np.repeat(g_mat, 2, axis=0)
g_mat = np.repeat(g_mat, 2, axis=1)
print("printing g_mat after 2nd repeate")
print(g_mat)


# Make H block diagonal
spin_ind = np.arange(H.shape[0], dtype=np.int) % 2
H *= (spin_ind.reshape(-1, 1) == spin_ind)
g_mat *= (spin_ind.reshape(-1, 1) == spin_ind)

print("after spin blocking")
print(H)
print(g_mat)

print('..finished transformation in %.3f seconds.\n' % (time.time() - t))

Here we are going to generate the set of all singly-excited determinants $|\Phi_i^a\rangle$
that we will call `singlesDets` and the reference determinant $|\Phi_0\rangle$ 
that we will calle `refDet`.

In [ ]:


print('Generating singly-excited determinants')
t = time.time()
singlesDets = []
for alpha in combinations(range(nmo), ndocc):
    alpha_ex_level = compute_excitation_level(alpha, ndocc)
    for beta in combinations(range(nmo), ndocc):
        beta_ex_level = compute_excitation_level(beta, ndocc)
        if alpha_ex_level + beta_ex_level == 1:
            print(F' adding alpha: {alpha} and beta: {beta}\n') 
            singlesDets.append(Determinant(alphaObtList=alpha, betaObtList=beta))

print('..finished generating singly-excited determinants in %.3f seconds.\n' % (time.time() - t))
print(F'..there are {len(singlesDets)} determinants \n')
for i in range(len(singlesDets)):
    print(singlesDets[i])
    
print('Generating reference determinant')

occList = [i for i in range(ndocc)]
refDet = Determinant(alphaObtList=occList, betaObtList=occList)
print(refDet)


##### this illustrates some functionality of the determinant class.
numDet = len(detList)
print(numDet)
for i in range(numDet):
    for j in range(numDet):
        unique1, unique2, sign = detList[i].getUniqueOrbitalsInMixIndexListsPlusSign(detList[j])
        print(detList[i], detList[j], unique1, unique2, sign)
        


First we are going to generate the ${\bf A + \Delta} + n\hbar{\bf \Omega}$ block of the Hamiltonian
which results from $\langle \Phi_i^a,n | \hat{H} | \Phi_j^b,n\rangle$ terms.
We can compute ${\bf A + \Delta}$ once and then add it to the appropriate block of
the Hamiltonian matrix with the appropriate block of $n\hbar{\bf \Omega}$. 

In [ ]:
print('Generating A+\Delta sublock...')
A_D_generator = HamiltonianGenerator(H, MO)


A_D_matrix = A_D_generator.generateMatrix(singlesDets)
print("printing H")
print(A_D_matrix)

Next we are going to generate the ${\bf g}$ blocks which arise from $ -\sqrt{\frac{\omega}{2}}  \langle \Phi_i^a,n | {\bf \lambda} \cdot {\bf \hat{\mu}} (\hat{b}^{\dagger} + \hat{b}) | \Phi_j^b,m\rangle$

In [ ]:
print('Generating g sublock...')
g_generator = HamiltonianGenerator(g_mat, MO * 0)

G_matrix = g_generator.generateMatrix(singlesDets)
print("printing g")
print(G_matrix)

Next we are going to generate the ${\bf G}$ blocks which arise from $ -\sqrt{\frac{\omega}{2}}  \langle \Phi_0,n | {\bf \lambda} \cdot {\bf \hat{\mu}} (\hat{b}^{\dagger} + \hat{b}) | \Phi_j^b,m\rangle$

In [ ]:
n_s = len(singlesDets)
g_vec = np.zeros((1,n_s))
for i in range(n_s):
    g_vec[0,i] = g_generator.calcMatrixElement(refDet, singlesDets[i])
print(g_vec)
g_dag = np.conj(g_vec).T
print(g_dag)

Now generate ${\bf \Omega}$ 

In [ ]:
Omega_matrix = np.identity(n_s) * omega_val
print(Omega_matrix)

Generate $E_0$

In [ ]:
E_0 = A_D_generator.calcMatrixElement(refDet, refDet)
print(E_0)

Now build the entire matrix

In [ ]:
# define the offsets
R0_offset = 0
S0_offset = 1
R1_offset = n_s + 1
S1_offset = n_s + 2


H_CISS_PF = np.zeros((n_s * 2 + 2, n_s * 2 + 2))
H_label = np.zeros((n_s * 2 + 2, n_s * 2 + 2),dtype=str)

H_CISS_PF[R0_offset, R0_offset] = E_0
H_label[R0_offset, R0_offset] ='E'
H_CISS_PF[R0_offset:S0_offset, S1_offset:] = g_vec
H_label[R0_offset:S0_offset, S1_offset:] = 'g'


H_CISS_PF[S0_offset:R1_offset, R1_offset:S1_offset] = g_dag
H_label[S0_offset:R1_offset, R1_offset:S1_offset] = 'g'

H_CISS_PF[R1_offset:S1_offset, S0_offset:R1_offset] = g_vec 
H_label[R1_offset:S1_offset, S0_offset:R1_offset] = 'g'

H_CISS_PF[S1_offset:,          R0_offset:S0_offset] = g_dag
H_label[S1_offset:,          R0_offset:S0_offset] = 'g'

H_CISS_PF[S0_offset:R1_offset, S0_offset:R1_offset] = A_D_matrix
H_label[S0_offset:R1_offset, S0_offset:R1_offset] = 'a'

H_CISS_PF[R1_offset, R1_offset] = E_0 + omega_val
H_label[R1_offset, R1_offset] = 'o'


H_CISS_PF[S1_offset:, S1_offset:] = A_D_matrix + Omega_matrix
H_label[S1_offset:, S1_offset:] = 'A'

H_CISS_PF[S1_offset:,S0_offset:R1_offset] = G_matrix
H_label[S1_offset:,S0_offset:R1_offset] = 'G'

H_CISS_PF[S0_offset:R1_offset, S1_offset:] = G_matrix
H_label[S0_offset:R1_offset, S1_offset:] = 'G'

print("Printing the Hamiltonian schematic")
print(H_label)

print("Printing the Hamiltonian matrix")
print(H_CISS_PF)



In [ ]:
print('..finished generating Matrix in %.3f seconds.\n' % (time.time() - t))

print('Diagonalizing Hamiltonian Matrix...')

t = time.time()
#print(E_0)
e_fci, wavefunctions = np.linalg.eigh(H_CISS_PF)
#print(e_fci)
for i in range(len(e_fci)):
    ex_e = e_fci[i]-E_0
    print(F'{ex_e:.12e}')

In [ ]:
print('..finished diagonalization in %.3f seconds.\n' % (time.time() - t))
for i in range(0,len(e_fci)):
    en = e_fci[i] + mol.nuclear_repulsion_energy() + dc - cqed_scf_e
    print(F'{en}\n')
fci_mol_e = e_fci[3] + mol.nuclear_repulsion_energy() + dc

print('# Determinants:     % 16d' % (len(singlesDets)))

print('SCF energy:         % 16.10f' % (scf_e))
print('FCI correlation:    % 16.10f' % (fci_mol_e - scf_e))
print('Total FCI energy:   % 16.10f' % (fci_mol_e))

In [ ]:
assert np.isclose(0.71721636, 0.7172163598047324, 1e-7)

In [ ]:
print(dc)
print(Hamiltonian_matrix)

In [ ]:
print(H)

In [ ]:
print(MO)

In [ ]:
# Create instance of MintsHelper class
#mints = psi4.core.MintsHelper(wfn.basisset())
# 2 electron integrals in CQED-RHF basis
Ca = wfn.Ca()
#MO_spin = np.asarray(mints.mo_spin_eri(Ca, Ca))

MO_spat = np.asarray(mints.mo_eri(Ca, Ca, Ca, Ca))


In [ ]:
nso = 2 * nmo
MO_spin = np.zeros((nso, nso, nso, nso))
for i in range(nso):
    for j in range(nso):
        for k in range(nso):
            for l in range(nso):
                MO_spin[i, j, k, l] = map_spatial_to_spin(MO_spat, i, j, k, l)
                
                
    
    

In [ ]:
assert np.allclose(MO_spin, MO)